In [18]:
!pip install -q yfinance
!pip install -q matplotlib
!pip install -q pandas_datareader


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\theaw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\theaw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\theaw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [172]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import datetime
from dateutil.relativedelta import relativedelta


# Money Supply

In [191]:
def fetch_money_supply():
    # Define the list of money supply series to fetch
    money_supply_series = ['M1SL', 'M2SL']

    # Fetch money supply data using pandas_datareader
    money_supply_data = pdr.get_data_fred(money_supply_series, start_date_with_shift, end_date)

    # Calculate the YoY percentage change
    money_supply_data['M1SL_YoY'] = money_supply_data['M1SL'].pct_change(12) * 100
    money_supply_data['M2SL_YoY'] = money_supply_data['M2SL'].pct_change(12) * 100


    return money_supply_data


In [192]:
# Retrieves all MS between start and end date and returns MS differences in each period bin
def money_supply_info(macro_indicators):
    money_supply_data = fetch_money_supply()
    
    money_supply_data_average = money_supply_data.resample(period_length).mean()

    money_supply_data_average['M1-M2'] = money_supply_data_average['M1SL_YoY'] - money_supply_data_average['M2SL_YoY']

    macro_indicators = pd.merge(macro_indicators, money_supply_data_average['M1-M2'], on='DATE', how='left')
    return macro_indicators

# CPI Inflation

In [193]:
def fetch_cpi_annual_change():
    try:
        # Fetch U.S. Consumer Price Index (CPI) data from FRED
        cpi_data = pdr.get_data_fred('CPIAUCSL', start = start_date_with_shift, end = end_date)

        if not cpi_data.empty:
            # Calculate annual percentage change
            cpi_data['Annual_Change'] = cpi_data['CPIAUCSL'].pct_change(12) * 100

            return cpi_data[['Annual_Change']]
        else:
            return None  # Return None if there is no data
    except Exception as e:
        return None  # Return None if there is an error


In [194]:
def cpi_info(macro_indicators):
    cpi_data_info = pd.DataFrame()
    cpi_data = fetch_cpi_annual_change()
    cpi_data_info['average_cpi_change'] = cpi_data.resample(period_length).mean()
    cpi_data_info['single_average_cpi_change'] = cpi_data_info['average_cpi_change'] - cpi_data_info['average_cpi_change'].shift(1)
    cpi_data_info['lag_single_average_cpi_change'] = cpi_data_info['single_average_cpi_change'].shift(1)

    macro_indicators = pd.merge(macro_indicators, cpi_data_info[['average_cpi_change', 'single_average_cpi_change', 'lag_single_average_cpi_change']], on = 'DATE', how = 'left')
    
    return(macro_indicators)

# Unemployment

In [201]:
def fetch_unemployment_data():
    try:
        # Define the ticker symbol for the U.S. unemployment rate (UNRATE)
        unemployment_ticker = "UNRATE"

        # Fetch unemployment data from FRED
        unemployment_data = pdr.get_data_fred(unemployment_ticker, start = start_date_with_shift, end = end_date)

        return unemployment_data
    except Exception as e:
        return None  # Return None if there is an error

In [216]:
def unemployment_info(macro_indicators):
    unemployment_info = pd.DataFrame()
    unemployment_data = fetch_unemployment_data()
    unemployment_info['average_unemployment'] = unemployment_data.resample(period_length).mean()
    unemployment_info['average_unemployment_change'] = unemployment_data['average_unemployment'] - unemployment_data['average_unemployment'].shift(1)
    unemployment_info['lag_average_unemployment_change'] = unemployment_data['average_unemployment'].shift(1)
    macro_indicators = pd.merge(macro_indicators, unemployment_info[['average_unemployment', 'average_unemployment_change', 'lag_average_unemployment_change']])
    return macro_indicators

# Interest Rates

In [195]:
def fetch_fed_interest_rate():
    try:
        # Fetch Federal Reserve interest rate data (FEDFUNDS) from FRED
        fed_interest_rate = pdr.get_data_fred('FEDFUNDS', start = start_date_with_shift, end=end_date)

        if not fed_interest_rate.empty:
            return fed_interest_rate
        else:
            return None  # Return None if there is no data
    except Exception as e:
        return None  # Return None if there is an error



In [196]:
def fed_interest_rate_info(macro_indicators):
    fed_interest_rate_info = pd.DataFrame()
    fed_interest_rate = fetch_fed_interest_rate()
    fed_interest_rate_info['average_interest_rate'] = fed_interest_rate.resample(period_length).mean()
    fed_interest_rate_info['single_rate_period_change'] = fed_interest_rate_info['average_interest_rate'] - fed_interest_rate_info['average_interest_rate'].shift(1)
    fed_interest_rate_info['lag_single_rate_change'] = fed_interest_rate_info['single_rate_period_change'].shift(1)
    macro_indicators = pd.merge(macro_indicators, fed_interest_rate_info[['average_interest_rate','single_rate_period_change', 'lag_single_rate_change']],on = 'DATE', how = 'left')
    return(macro_indicators)



# 

# MAIN

In [217]:
# Main
end_date = datetime.datetime(2022, 1, 1)
start_date = end_date - relativedelta(months = 36)
start_date_with_shift = start_date - relativedelta(months = 15)

period_length = '1M'
date_range = pd.date_range(start=start_date, end=end_date, freq=test_period_length)
macro_indicators = pd.DataFrame({'DATE': date_range})

# Get M1S-M2S info
macro_indicators = money_supply_info(macro_indicators)

# Get interest rate info
macro_indicators = fed_interest_rate_info(macro_indicators)

# Get CPI info
macro_indicators = cpi_info(macro_indicators)

# Get unemployment info
macro_indicators = unemployment_info(macro_indicators)





KeyError: 'average_unemployment'

In [ ]:
macro_indicators

# MAIN

In [214]:
macro_indicators.columns

#lgbm ML model

Index(['DATE', 'M1-M2', 'average_interest_rate', 'single_rate_period_change',
       'lag_single_rate_change', 'average_cpi_change',
       'single_average_cpi_change', 'lag_single_average_cpi_change'],
      dtype='object')